In [1]:
%pip install tslearn
import tslearn

  Using cached numpy-2.3.5-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 10.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.2/37.2 MB 10.1 MB/s eta 0:00:00a 0:00:01
Using cached numpy-2.3.5-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.2
    Uninstalling numpy-2.4.2:
      Successfully uninstalled numpy-2.4.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [tslearn]m3/4 [tslearn]]

[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

In [ ]:
from data import Data

# Get all data in pandas dataframe
interest_stocks = ["NVDA", "GOOGL", "AAPL", "MSFT", "AMZN", "META"]
all_data = Data.get_symbols_returns(symbols=interest_stocks, length=250)
print(all_data)

            date symbol  day_returns
0     2025-01-02   NVDA     0.029935
1     2025-01-03   NVDA     0.044538
2     2025-01-06   NVDA     0.034332
3     2025-01-07   NVDA    -0.062170
4     2025-01-08   NVDA    -0.000214
...          ...    ...          ...
1495  2025-12-24   META     0.003925
1496  2025-12-26   META    -0.006382
1497  2025-12-29   META    -0.006935
1498  2025-12-30   META     0.011022
1499  2025-12-31   META    -0.008799

[1500 rows x 3 columns]


In [7]:
import numpy as np

all_data = all_data.sort_values(["symbol", "date"])
pivot = all_data.pivot(index="symbol", columns="date", values="day_returns")
print(pivot)

date    2025-01-02  2025-01-03  2025-01-06  ...  2025-12-29  2025-12-30  2025-12-31
symbol                                      ...                                    
AAPL     -0.026236   -0.002009    0.006739  ...    0.001317   -0.002484   -0.004468
AMZN      0.003783    0.018027    0.015255  ...   -0.001935    0.001982   -0.007354
GOOGL     0.000687    0.012458    0.026487  ...    0.000159    0.000925   -0.002708
META      0.023450    0.008995    0.042290  ...   -0.006935    0.011022   -0.008799
MSFT     -0.006928    0.011396    0.010630  ...   -0.001251    0.000780   -0.007918
NVDA      0.029935    0.044538    0.034332  ...   -0.012124   -0.003613   -0.005545

[6 rows x 250 columns]


In [10]:
import numpy as np

model_inputs = pivot.values[:, :, np.newaxis]
print(model_inputs.shape)

(6, 250, 1)


In [14]:
# Scaling time series using tslearn TimeSeriesScalerMeanVariance
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

scaler = TimeSeriesScalerMeanVariance()
model_inputs_scaled = scaler.fit_transform(model_inputs)

In [18]:
from tslearn.clustering import TimeSeriesKMeans

model = TimeSeriesKMeans(
    n_clusters=3,
    metric="dtw",
    max_iter=25,
    metric_params={"sakoe_chiba_radius": 5},  # speed constraint
    random_state=0,
    n_jobs=-1
)

labels = model.fit_predict(model_inputs_scaled)

/Users/saake/Documents/Personal Development/cerebrum/venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:95: FutureWarning: Function stable_cumsum is deprecated; `sklearn.utils.extmath.stable_cumsum` is deprecated in version 1.8 and will be removed in 1.10. Use `np.cumulative_sum` with the desired dtype directly instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
cluster_map = pivot.index.to_series().to_frame(name="stock")
cluster_map["cluster"] = labels
print(cluster_map)

        stock  cluster
symbol                
AAPL     AAPL        2
AMZN     AMZN        2
GOOGL   GOOGL        1
META     META        2
MSFT     MSFT        0
NVDA     NVDA        0
